In [ ]:
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import collections
import random
import re
import numpy as np
import pandas as pd
import os
import time
import json
from glob import glob
from PIL import Image
import pickle

from tqdm.auto import tqdm
tqdm.pandas()

from pathlib import Path

import Levenshtein

import csv

In [ ]:
rootpath = "/data/"

In [ ]:
train = pd.read_csv(rootpath + "train_labels.csv")

In [ ]:
def get_train_file_path(image_id):
    return rootpath + "train/{}/{}/{}/{}.png".format(
        image_id[0], image_id[1], image_id[2], image_id 
    )

train['file_path'] = train['image_id'].apply(get_train_file_path)

print(f'train.shape: {train.shape}')
display(train.head())

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [ ]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [ ]:
def output_path(path):
    outpath = path.replace(".png", ".npy").replace("train", "train_inception")
    path = "/".join(outpath.split("/")[:-1])
    Path(path).mkdir(parents=True, exist_ok=True)
    return outpath

In [ ]:
# Get unique images
encode_train = train["file_path"].values

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(16)

for img, path in image_dataset:
  batch_features = image_features_extract_model(img)
  batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    outpath = output_path(path_of_feature)
    np.save(outpath, bf.numpy())